In [1]:
import os
os.chdir("/home/yangkai/codegen-detection/")

In [2]:
import pandas as pd
df = pd.read_pickle("dataset/mbpp_gen_test.pkl")[['problem_id','solution','prompt','extracted_full_func']]
df_test = pd.read_json("dataset/mbpp_with_test.json",lines=True)
df_test = df_test[['problem_id','test_list','test_setup_code']]

In [3]:
import re
for_test_df = pd.merge(df,df_test,on='problem_id',how='left')

def extract_function_name(code):
    pattern = r"def\s+([\w_]+)\s*\("
    function_names = re.findall(pattern, code)
    return function_names

gen_names = for_test_df.extracted_full_func.apply(extract_function_name)
orig_names = for_test_df.solution.apply(extract_function_name).tolist()
tests = for_test_df.test_list.tolist()

filter_orig_names = []
for names,test in zip(orig_names,tests):
    for name in names:
        if name in test[0]:
            filter_orig_names.append(name)
            break

for_test_df['entry_point_name'] = filter_orig_names
for_test_df['gen_names'] = gen_names

In [4]:
from excution import check_correctness
import tqdm

correct_ids = []
bar = tqdm.tqdm(total=len(for_test_df))
for _,p in for_test_df.iterrows():
    input = {'prompt':'','test':"\n".join(p['test_list']),'task_id':p['problem_id']}
    completion = p['extracted_full_func']
    entry_point = p['entry_point_name']
    is_pass = False
    for gen_name in p['gen_names']:
        norm_completion = completion.replace(gen_name,entry_point)
        res = check_correctness(input,norm_completion,5)
        if res['passed']:
            is_pass = True
            break
    
    if is_pass:
        correct_ids.append(p['problem_id'])
    bar.update(1)

 98%|█████████▊| 229/233 [00:38<00:00, 16.98it/s]

In [11]:
df = pd.read_pickle("dataset/mbpp_gen_test.pkl")
correct_df = df[df['problem_id'].isin(correct_ids)]
correct_df.to_pickle("dataset/mbpp_gen_test_correct.pkl")